In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [2]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('accuracy') > 0.999):
        print("\nReached 99.9% accuracy so cacelling training!\n")
        self.model.stop_training = True

my_callback = myCallback()

--2020-08-18 09:38:05--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 216.58.200.16, 216.58.197.112, 172.217.31.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|216.58.200.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  8.94MB/s    in 0.3s    

2020-08-18 09:38:05 (8.94 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [3]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', 
              optimizer=RMSprop(learning_rate=0.0001),
              metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18496)             0

In [5]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255.)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',
        target_size=(150,150),
        batch_size=8,
        class_mode='binary')   # classes=['sad', 'happy']

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [6]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
        train_generator,
        steps_per_epoch=5,
        epochs=15,
        verbose=1,
        callbacks=[my_callback])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
5/5 [==============================] - 0s 19ms/step - loss: 0.7900 - accuracy: 0.5750
Epoch 2/15
5/5 [==============================] - 0s 20ms/step - loss: 0.4969 - accuracy: 0.8750
Epoch 3/15
5/5 [==============================] - 0s 20ms/step - loss: 0.4238 - accuracy: 0.8250
Epoch 4/15
5/5 [==============================] - 0s 18ms/step - loss: 0.3179 - accuracy: 0.9250
Epoch 5/15
5/5 [==============================] - 0s 18ms/step - loss: 0.2564 - accuracy: 0.9000
Epoch 6/15
5/5 [==============================] - 0s 18ms/step - loss: 0.2382 - accuracy: 0.9500
Epoch 7/15
5/5 [==============================] - 0s 18ms/step - loss: 0.2993 - accuracy: 0.8500
Epoch 8/15
5/5 [==============================] - 0s 19ms/step - loss: 0.1719 - accuracy: 0.9500
Epoch 9/15
5/5 [==============================] - 0s 18ms/step - loss: 0.1208 - accuracy: 0.9750
Epoch 10/15
5/5 [==============================] - 0s 19ms/step - loss: 0.1305 - accuracy: 0.9750
Epoch 11/15
4/5 [============